In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from pathlib import Path
import csv
import hvplot.pandas
import sqlalchemy as sql
import seaborn as sns
from matplotlib import pyplot as plt
import datetime as dt
from sqlalchemy import create_engine, inspect 
from tkinter import *
%matplotlib inline
from db_utils import get_debt_free_db_engine, get_transaction_for_user_year, get_account_for_user

In [ ]:
# Create an engine to interact with the database
engine = get_debt_free_db_engine()

In [ ]:
transaction_2020 = get_transaction_for_user_year(engine, 1, '2020')

In [ ]:
transaction_2020['year'] = pd.DatetimeIndex(transaction_2020['transaction_date']).year

In [ ]:
transaction_2020['month'] = pd.DatetimeIndex(transaction_2020['transaction_date']).month

In [ ]:
transaction_2020.head()

In [ ]:
summary_transaction_2020 = pd.DataFrame(round(transaction_2020.groupby(['user_id', 'card','month'], as_index=False)[('amount')].sum(),2))
summary_transaction_2020

In [ ]:
account_data = get_account_for_user(engine,1)

In [ ]:
account_data

In [ ]:
card= account_data.loc[account_data['card'] == 1]
annual_percent_rate= float(card['annual_percentage_rate'])
annual_percent_rate

In [ ]:
## Function to calculate new balance for current billing period
def calc_new_balance(row):
    result = {}
    """ row: table row containing 'user_id','card','month','amount' columns
        annual_percent_rate: interest calculated
    """
   
    # calculate monthly percent rate
    monthly_percent_rate = float((annual_percent_rate/100) / 12)
        
    # assign remaining balance
    monthly_remaining_balance = row['amount']
    
    # calculate minimum payment
    minimum_payment = (monthly_remaining_balance) * 0.02
    row['minimum_payment'] = round(minimum_payment,2)
    
    # deduct payment made i.e. min payment
    adjusted_balance = monthly_remaining_balance - minimum_payment
    
    # calculate interest
    interest_charged = adjusted_balance * monthly_percent_rate
    row['interest_charged'] = round(interest_charged,2)
    
    # calculate new balance
    new_balance = adjusted_balance + interest_charged
    row['new_balance'] = round(new_balance,2)

    return row

In [ ]:
## Apply function to each row of dataframe
summary_transaction_2020 = summary_transaction_2020.apply(lambda row: calc_new_balance(row), axis =1)
summary_transaction_2020